# Reinforcement learning - 4 op een rij

Reinforcement learning komt uit de studie van Markov Chains of Processen voor.
Dit is een random opeenvolging van states waarbij elke transisitie een mogelijke kans heeft.
Door een reward te koppelen aan elke state waarin je komt kan je een functie opstellen die de de totale reward maximaliseert.
Dit is het basisidee achter reinforcement learning.

Een aantal belangrijke termen/concepten hierbij zijn:
* De agent
* Het environment
* De state space
* De action space
* De reward en return
* Exploration vs exploitation

## Q-learning

Een eerste algoritme dat we bekijken voor reinforcement learning uit te voeren is Q-learning.
Dit algoritme maakt gebruik van de Q-functie of action-value function.
Hiervoor houdt het Q-learning algoritme een matrix bij dat de reward van actie in een state bepaald.
In een verkenningsfase laten we toe dat er sub-optimale keuzes genomen worden.
Nadat dit lang genoeg gerund heeft, gaan we over naar een exploitation fase waarbij enkel de beste keuzes genomen worden.

Om te tonen hoe je het Q-learning algoritme kan implementeren, kan je gebruik maken van het [gymnasium package](https://gymnasium.farama.org/).
Dit bevat heel wat eenvoudige environments van spelletjes in python die hiervoor gebruikt kunnen worden.

In onderstaande code gaan we een AI-model maken om vier op een rij te spelen.
Het environment (met reeds een aantal ingebouwde AI-agents) kan je [hier](https://github.com/lucasBertola/Connect-4-Gym-env-Reinforcement-learning/tree/main) vinden.
In onderstaande code-cell toon ik een demo van hoe je zelf tegen een Agent 4 op een rij kan spelen.

In [ ]:
from connect_four_gymnasium import ConnectFourEnv
from connect_four_gymnasium.players import SelfTrained6Player
from connect_four_gymnasium.players import ConsolePlayer
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

you = ConsolePlayer()
env = ConnectFourEnv(opponent= SelfTrained6Player(deteministic=True), render_mode="rgb_array",main_player_name="you")

obs , _=  env.reset()
for i in range(5000):
    
    clear_output(wait=True)
    frame = env.render()
    plt.imshow(frame)
    plt.axis('off')
    display(plt.gcf())
    plt.close()  # close the figure to avoid memory buildup
    
    action = you.play(obs)
    obs, rewards, done, truncated,info = env.step(action)

    print(obs)
    print(rewards)
    
    if(truncated or done):
        if rewards > 0:
            print("player won")
            break
        else:
            print("player lost")
            break
        obs , _=  env.reset()

Zoals reeds eerder besproken hebben we bij reinforcement learning hebben we de volgende zaken nodig om een AI model te trainen:
* De mogelijke acties die het AI-model kan nemen (action space)
* De state van de environment dat gedetecteerd wordt door het model (observation space)
* De beloningen uitgestuurd door het systeem

Wat deze zaken voorstellen en welke structuur deze waarden hebben hangt sterk af van de specifieke omgeving en kan dus vaak opgezocht worden in de documentatie van de omgeving.
Door het uitprinten van de observatie en de reward hebben we reeds een indicatie van wat de omgeving/environment bijhoudt als interne state en uitstuurt als beloning.

In dit geval hebben we de volgende kenmerken:
* action space: een getal tussen 1 en 7 (welke kolom je een stuk inplaatst)
* observation space: een matrix van 6x7 met 0 als de plaats vrij is, 1 en -1 als respectievelijk speler 1 of speler 2 er een munt geplaatst heeft
* rewards: 0 als het spel bezig is, 1 als speler 1 gewonnen heeft (of speler 2 een ongeldige zet gedaan heeft) en -1 als speler 2 gewonnen heeft (of speler 1 een ongeldige zet gedaan heeft).

## Oefening - Random agent

Schrijf een AI-agent die random zetten neemt. Hiervoor schrijf je een klasse RandomAgent die een subklasse is van de base-klasse Player in de library.
De base klasse heeft de volgende structuur:
```python
class Player:
    def __init__(self, name):
        self.name = name

    def play(self, observation):
        raise NotImplementedError("The 'play' method must be implemented in the child class")

    def getElo(self):
        return None
    
    def getName(self):
        return self.name

    def isDeterministic(self):
        raise NotImplementedError("The 'isDeterministic' method must be implemented in the child class")
```

Zorg er daarna voor dat je een spel kan spelen tegen je eigen geschreven agent.

Op dit moment kunnen we deze agent een aantal keer 4 op een rij laten spelen tegen een andere speler.
Doe dit hieronder en bekijk het winstpercentage tegen verschillende opponenten.
Ga in de documentatie op zoek naar een aantal tegenstanders

## Van random agent naar lerende agent

Maak nu een andere agent die het Q-learning algoritme implementeert.
Hiervoor zorgen we dus voor de volgende zaken:
* Zorg dat er een dictionary is voor (state, action) -> q_value
* De gewenste actie voor een gegeven state/observation is dan degene met de hoogste waarde in de dictionary. Indien er meerdere eenzelfde waarde hebben kan je kiezen
* Voeg een learn functie toe dat de reward update volgens de functie waarbij alpha de learning_rate is en gamma de discount_factor
```
        Q(s,a) ← Q(s,a) + α [ r + γ max_a' Q(s',a') – Q(s,a) ]
```
* Voeg een epsilon-greedy aanpak toe in het nemen van de actie om met een bepaalde factor een random waarde te kiezen en niet de beste
* Voeg een set_deterministic functie toe om de episolon-waarde op 0 te zetten zodat er geen exploration meer toegelaten wordt

Nu moeten we een leerproces maken. Hiervoor gaan we onze Q-learning agent laten spelen.
Hier zijn verschillende opties voor  

1. **Agent traint tegen zichzelf (self-play)**

✅ Voordeel: de agent leert steeds sterker spelen, omdat hij moet verbeteren om zichzelf te verslaan.

❌ Nadeel: kan instabiel zijn → beide kanten maken dezelfde fouten in het begin, en de agent kan "domme" strategieën aanleren die in echte tegenstand niet werken.

Self-play werkt vaak beter als je af en toe het oude beleid (policy) behoudt, of een mix doet (bijvoorbeeld: soms random, soms het huidige beleid).

2. **Agent traint tegen een RandomPlayer**

✅ Voordeel: simpele manier om te starten, de agent leert al snel dat domme zetten verliezen opleveren.

✅ Makkelijk om eerste progressie te zien in de Q-waarden.

❌ Nadeel: na een tijdje leert de agent vooral exploiteren dat de tegenstander random speelt, en leert niet per se een sterke algemene strategie.

3. **Agent traint tegen een bestaande sterke speler (bv. je SelfTrained6Player)**

✅ Voordeel: de agent wordt gedwongen om tegen "echte" strategieën te leren.

❌ Nadeel: in het begin verliest hij bijna alles → weinig beloning → learning kan traag of zelfs falen.
→ vaak los je dit op door curriculum learning: eerst tegen RandomPlayer, dan tegen sterkere tegenstanders.

Wij gaan de laatste methode toepassen waarbij opvolgend de BabySmarterPlayer, de ChildPlayer, de ChildSmarterPlayer, de TeenagerPlayer. de TeenagerSmarterPlayer. de AdultPlayer en de AdultSmarterPlayer gebruikt worden.
Elke keer dat de ai 5 opeenvolgende keren wint van de tegenstander gaat hij over naar de volgende moeilijkheid.
In het geval hij tegen iedereen gewonnen is, dan speelt hij verder tegen zichzelf.
Schrijf nu een trainingslus die dit uitvoert en train de agent.

Speel nu zelf nog eens tegen de agent, hoe ervaar je het?

## RL in neural networks

Het gebruik van Q-learning werkt goed als het aantal states en acties beperkt zijn.
Dit is echter zelden het geval, denk bijvoorbeeld aan een continue variabele zoals snelheid of locatie.

Een oplossing hiervoor is om de action-value functie die in Q-learning geoptimaliseerd wordt te benaderen ipv exact te berekenen.
Dit kan bijvoorbeeld door middel van een neuraal netwerk te gebruiken.
Er zijn verschillende model-structuren die hiervoor ontwikkeld zijn zoals:
- DQN (onderwerp van onderstaande demo)
- REINFORCE
- DDPG
- TD3
- PPO
- SAC

Voor we beginnen met het uitwerken van een model.
Bekijk [deze tutorial](https://www.tensorflow.org/agents/tutorials/1_dqn_tutorial) en beantwoord de volgende vragen:
- Wat is de state en wat zijn de mogelijke acties?
- Wat is de structuur van het gebruikte DQN?
- Zijn er nieuwe hyperparameters gebruikt?
- Welke metriek wordt er gebruikt en waar wordt deze berekend?
- Hoe worden de gewichten aangepast?
- Waarvoor wordt de ReplayBuffer gebruikt?

**Antwoord:**

Schrijf nu zelf de nodige code om het DQN-model toe te passen op het "4 op een rij" environment van hierboven.

Speel tenslotte nog eens zelf tegen je agent